In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt

import cv2
import tifffile

In [5]:
file_list = os.listdir('data/Fluo-C2DL-Huh7/01')
mask_path = 'data/Fluo_mask_data'
save_path = 'data/Fluo_single_cell_data'

bias = 3

if not os.path.exists(save_path):
    os.makedirs(save_path)

for file in file_list:
    filename = file.split('.')[0]
    print('Processing ' + file + '...')
    img = tifffile.imread('data/Fluo-C2DL-Huh7/01/' + file)
    print(img.shape)
    img = np.transpose(img, (2, 1, 0))
    mask = np.load(mask_path + '/' + '01' + filename + '_mask.npy')
    num_cells = np.max(mask.flatten())
    # plt.subplot(1, 2, 1)
    # plt.imshow(img[0], cmap='gray')
    # plt.subplot(1, 2, 2)
    # plt.imshow(mask, cmap='gray')
    # plt.show()
    # print(min(mask.flatten()), max(mask.flatten()))
    print('Number of cells in ' + file + ': ' + str(num_cells))
    # break
    for i in range(1, num_cells + 1):
        cell_index = np.argwhere(mask == i)
        # print(cell_index.shape)
        x, y = cell_index[:, 0], cell_index[:, 1]
        x_min, x_max = np.min(x), np.max(x)
        y_min, y_max = np.min(y), np.max(y)
        # print(x_min, x_max, y_min, y_max)
        if x_min < bias or x_max > img.shape[1] - bias or y_min < bias or y_max > img.shape[2] - bias:
            print('Cell ' + str(i) + ' in ' + file + ' is too close to the edge!')
            continue
        cell = img[:, y_min-bias:y_max+bias, x_min-bias:x_max+bias]
        # if i == 2:
        # # print(cell)
        #     plt.subplot(2, 2, 1)
        #     plt.imshow(cell[0])
        #     plt.subplot(2, 2, 2)
        #     plt.imshow(cell[1])
        #     plt.subplot(2, 2, 3)
        #     plt.imshow(cell[2])
        #     plt.subplot(2, 2, 4)
        #     plt.imshow(cell[3])
        #     plt.show()
        #     print(cell.shape)
        if (np.count_nonzero(cell[0]) / (cell.shape[1]*cell.shape[2])) < 0.5:
            print('Empty cell!')
            continue
        tifffile.imsave('data/single_cell_data/' + '01' + filename + '_' + str(i) + '.tif', cell)
        print('Cell ' + str(i) + ' in ' + file + ' saved!')
    #     break
    # break
        # np.save(save_path + '/' + filename + '_' + str(i) + '.npy', cell)

Processing t000.tif...
(1024, 1024)


ValueError: axes don't match array